# Coordinate Descent to Cluster Cells

We want to perform coordinate descent to cluster cells. We will initialize a clustering using a common method, such as spectral clustering. Then, we will use that clustering to fit maximum likelihood embryo centers. Next, we will set the cluster of each cell according to maximum likelihood. This will be repeated until convergence.

In [1]:
import pandas as pd
import anndata
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
from functools import partial
from multiprocessing import Pool
from sklearn.metrics import adjusted_rand_score
from itertools import product

## Load Data

In [2]:
NUM_EMBRYOS = 3

CALLS_SAVE_PATH = 'data/calls_2022_07_28_umi_filtered.h5ad'
READS_SAVE_PATH = 'data/reads_2022_07_28_umi_filtered.h5ad'

In [3]:
variants_joined = anndata.read_h5ad(CALLS_SAVE_PATH)
reads_joined = anndata.read_h5ad(READS_SAVE_PATH)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [4]:
inds = variants_joined.var.index

chrom = []
pos = []
transition = []

for ind in inds:
    split_ind = ind.split(':')
    chrom.append(split_ind[0])
    pos.append(split_ind[1])
    transition.append(split_ind[2])

In [5]:
counts = {position:0 for position in pos}

for position in pos:
    counts[position] += 1

## Run Descent

In [6]:
# Computes the log probability of each value of
# variant j in cluster i
def compute_best_log_prob_pi(j, i=None):
    # Get the calls for the variant for each cell in the cluster
    cell_calls = C[z == i, j]
    
    probs = []
    
    for pi_call in range(4):
        prob = 0
        
        for cell_call in cell_calls:
            if T[pi_call, cell_call] == 0:
                prob = -np.inf
                break
            
            prob += np.log(T[pi_call, cell_call])
            
        probs.append(prob)
        
    return np.argmax(probs)

# Sets pi by computing choosing the best log probability variant
# call for each cluster
def set_pi():
    pi = np.zeros((NUM_EMBRYOS, C.shape[1]), dtype=np.int8)
    
    for i in range(NUM_EMBRYOS):
        emb_prob_fun = partial(compute_best_log_prob_pi, i=i)
        
        p = Pool(NUM_THREADS)
        calls = list(tqdm(p.imap(emb_prob_fun, range(C.shape[1])), total=C.shape[1]))
        p.close()
                
        pi[i, :] = calls
            
    return pi

# Computes the log probabilities accross the possible clusters for
# C[i]
def compute_best_log_prob_C(i):
    probs = []
    
    for l in range(NUM_EMBRYOS):
        prob = 0
        
        for j in range(C.shape[1]):
            if T[pi[l, j], C[i, j]] == 0:
                prob = -np.inf
                break
                
            prob += np.log(T[pi[l, j], C[i, j]])
            
        probs.append(prob)
        
    return np.argmax(probs)


def get_row_counts(i):
    # Counts the number of transitions between what we think the actual embryo
    # value is (pi) and what we see in our clustered cell (C_i)
    count = np.zeros((4, 4))
    
    # Count the number of k1 -> k2 transitions
    for j in range(C.shape[1]):
        l = z[i]
        count[pi[l][j]][C[i][j]] += 1

    return count

def set_T():
    # Compute the most likely values for T by counting all the transitions    
    p = Pool(NUM_THREADS)
    counts = list(tqdm(p.imap(get_row_counts, range(C.shape[0])), total=C.shape[0]))
    p.close()
    
    T_new = np.zeros((4, 4))
    
    # Add together all cell counts
    for count in counts:
        T_new += count
        
    # Row normalize to make T a probability
    for i in range(4):
        row_sum = np.sum(T[i, :])
        
        if row_sum > 0:
            T[i, :] = T[i, :]/row_sum     
        
    return T_new

def set_z():    
    p = Pool(NUM_THREADS)
    z = np.array(list(tqdm(p.imap(compute_best_log_prob_C, range(C.shape[0])), total=C.shape[0])))
    p.close()
                            
    return z

In [7]:
NUM_THREADS = 90

def set_gt(epsilon):
    global C, z, T, pi
    
    # Our ground truth data
    C = variants_joined.X
    z = np.array(variants_joined.obs.spectral_clusters.copy())

    T = np.array([[1.        , 0.        , 0.        , 0.        ],
       [0.74778739, 0.25221261, 0.        , 0.        ],
       [0.95105229, 0.        , 0.04894771, 0.        ],
       [0.81110449, 0.0507234 , 0.11769942, 0.02047269]])

    T = T + epsilon
    T = T/T.sum(axis=1)
    
    pi = np.zeros((NUM_EMBRYOS, C.shape[1]), dtype=np.int8)

#0.935
# T = np.array([[0.8        , 0.0666        , 0.0666        , 0.0666        ],
#        [0.7, 0.2, 0.05       , 0.05        ],
#        [0.84580846, 0.070081273, 0.132586663, 0.047332063],
#        [0.7, 0.05    , 0.05        , 0.2]])
# T = np.random.rand(4, 4)

# for i in range(T.shape[0]):
#     T[i, :] = T[i, :]/np.sum(T[i, :])

# T = np.array([[0.6        , 0.1333      , 0.1333        , 0.1333       ],
#        [0.74778739, 0.25221261, 0.        , 0.        ],
#            [0.84580846, 0.03814512, 0.10065051, 0.01539591],
#            [0.74778739, 0.        , 0.        , 0.25221261]])

In [9]:
def find_opt():
    global pi, z
    
    score_prev = -1

    while 1:
        print('Setting pi')
        pi = set_pi()

        print('Setting z')
        z = set_z()

        score = adjusted_rand_score(variants_joined.obs.embryo, z)
        print('ARI is: ', score)

        if score-score_prev == 0:
            break

        score_prev = score
        
    return score

In [10]:
set_gt(0.01)
score = find_opt()

Setting pi


  0%|          | 0/27462 [00:00<?, ?it/s]

  0%|          | 0/27462 [00:00<?, ?it/s]

  0%|          | 0/27462 [00:00<?, ?it/s]

Setting z


...

  0%|          | 0/27462 [00:00<?, ?it/s]

  0%|          | 0/27462 [00:00<?, ?it/s]

Process ForkPoolWorker-1587:
Process ForkPoolWorker-1598:
Process ForkPoolWorker-1599:
Process ForkPoolWorker-1609:
Process ForkPoolWorker-1564:
Process ForkPoolWorker-1565:
Process ForkPoolWorker-1597:
Process ForkPoolWorker-1531:
Process ForkPoolWorker-1591:
Process ForkPoolWorker-1595:
Process ForkPoolWorker-1617:
Process ForkPoolWorker-1589:
Process ForkPoolWorker-1537:
Process ForkPoolWorker-1594:
Process ForkPoolWorker-1534:
Process ForkPoolWorker-1612:
Process ForkPoolWorker-1553:
Process ForkPoolWorker-1593:
Process ForkPoolWorker-1561:
Process ForkPoolWorker-1588:
Process ForkPoolWorker-1552:
Process ForkPoolWorker-1569:
Process ForkPoolWorker-1602:
Process ForkPoolWorker-1613:
Process ForkPoolWorker-1606:
Process ForkPoolWorker-1551:
Process ForkPoolWorker-1607:
Process ForkPoolWorker-1596:
Process ForkPoolWorker-1608:
Process ForkPoolWorker-1586:
Process ForkPoolWorker-1610:
Process ForkPoolWorker-1541:
Process ForkPoolWorker-1560:
Process ForkPoolWorker-1554:
Process ForkPo

KeyboardInterrupt: 

Process ForkPoolWorker-1544:
Process ForkPoolWorker-1574:
Process ForkPoolWorker-1567:
Process ForkPoolWorker-1618:
Process ForkPoolWorker-1562:
Process ForkPoolWorker-1540:
Process ForkPoolWorker-1601:
Process ForkPoolWorker-1533:
Process ForkPoolWorker-1600:
Process ForkPoolWorker-1578:
Process ForkPoolWorker-1539:
Process ForkPoolWorker-1577:
Process ForkPoolWorker-1576:
Process ForkPoolWorker-1549:
Process ForkPoolWorker-1581:
Process ForkPoolWorker-1566:
Process ForkPoolWorker-1611:
Process ForkPoolWorker-1619:
Process ForkPoolWorker-1605:
Process ForkPoolWorker-1545:
Process ForkPoolWorker-1568:
Process ForkPoolWorker-1616:
Process ForkPoolWorker-1536:
Process ForkPoolWorker-1592:
Process ForkPoolWorker-1543:
Process ForkPoolWorker-1535:
Process ForkPoolWorker-1547:
Process ForkPoolWorker-1538:
Process ForkPoolWorker-1550:
Process ForkPoolWorker-1571:
Process ForkPoolWorker-1620:
Process ForkPoolWorker-1579:
Process ForkPoolWorker-1583:
Process ForkPoolWorker-1556:
Process ForkPo

In [14]:
# Do a parameter sweep on epsilon
eps_range = np.linspace(0.01, 0.06, 5)
scores = []

for eps in tqdm(eps_range):
    # Reset the ground truth
    set_gt(eps)
    
    score = adjusted_rand_score(variants_joined.obs.embryo, z)
    print(f'Doing epsilon = {eps}, start score is {score}')
    print(z, T, pi)
    
    score = find_opt()
    scores.append(score)

  0%|          | 0/5 [00:00<?, ?it/s]

Doing epsilon = 0.01, start score is 0.49768641263750374
[2 2 2 ... 0 2 0] [[0.97115385 0.00961538 0.00961538 0.00961538]
 [0.72864172 0.25212751 0.00961538 0.00961538]
 [0.92408874 0.00961538 0.05668049 0.00961538]
 [0.78952355 0.05838788 0.1227879  0.02930066]] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Setting pi


  0%|          | 0/27462 [00:00<?, ?it/s]

  0%|          | 0/27462 [00:00<?, ?it/s]

  0%|          | 0/27462 [00:00<?, ?it/s]

...

  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run


  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._ar

  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self

  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-6-66454b2838e8>", line 52, in compute_best_log_prob_C
    prob += np.log(T[pi[l, j], C[i, j]])
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-6-66454b2838e8>", line 52, in compute_best_log_prob_C
    prob += np.log(T[pi[l, j], C[i, j]])
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/mnt/geofflab/jupyterhub/anaconda3/l

  File "<ipython-input-6-66454b2838e8>", line 52, in compute_best_log_prob_C
    prob += np.log(T[pi[l, j], C[i, j]])
KeyboardInterrupt
  File "<ipython-input-6-66454b2838e8>", line 52, in compute_best_log_prob_C
    prob += np.log(T[pi[l, j], C[i, j]])
KeyboardInterrupt
  File "<ipython-input-6-66454b2838e8>", line 52, in compute_best_log_prob_C
    prob += np.log(T[pi[l, j], C[i, j]])
  File "<ipython-input-6-66454b2838e8>", line 52, in compute_best_log_prob_C
    prob += np.log(T[pi[l, j], C[i, j]])
  File "<ipython-input-6-66454b2838e8>", line 48, in compute_best_log_prob_C
    if T[pi[l, j], C[i, j]] == 0:
  File "<ipython-input-6-66454b2838e8>", line 48, in compute_best_log_prob_C
    if T[pi[l, j], C[i, j]] == 0:
  File "<ipython-input-6-66454b2838e8>", line 52, in compute_best_log_prob_C
    prob += np.log(T[pi[l, j], C[i, j]])
KeyboardInterrupt
  File "<ipython-input-6-66454b2838e8>", line 52, in compute_best_log_prob_C
    prob += np.log(T[pi[l, j], C[i, j]])
  File "<ipython

In [1]:
pi

NameError: name 'pi' is not defined